# Parsing Text with TextFSM

In this short tutorial, we're going to first use the Netmiko library to gather some command output from a HPE FlexFabric 5900 switch and then use the TextFSM library to convert that into a python friendly data structure. 

In [1]:
import textfsm as textfsm
from netmiko import ConnectHandler



In [2]:
FF5900 = { 'device_type' : 'hp_comware', 'ip': '10.20.10.10', 'username' : 'admin', 'password': 'password'}

In [3]:
net_connect = ConnectHandler(**FF5900)
output = net_connect.send_command("display arp")

In [4]:
print(output)

  Type: S-Static   D-Dynamic   O-Openflow   R-Rule   M-Multiport  I-Invalid
IP address      MAC address    VLAN     Interface                Aging Type 
10.20.10.1      001e-c1dc-fc01 N/A      M-GE0/0/0                12    D    
10.20.1.254     001e-c1dc-fc01 1        XGE1/0/48                20    D    


In [5]:
with open('./dis_arp.template', 'r') as file:
    template = textfsm.TextFSM(file)

In [6]:
print(template)

Value IPADDRESS (\d+.\d+.\d+.\d+)
Value MACADDRESS (\w+-\w+-\w+)
Value VLAN (\S+|\d+)
Value INTERFACE (\S+)
Value AGING (\d+)
Value TYPE (\S+)

Start
  ^${IPADDRESS}\s+${MACADDRESS}\s+${VLAN}\s+${INTERFACE}\s+${AGING}\s+${TYPE} -> Record



In [8]:
my_arps = template.ParseText(output)

In [9]:
my_arps

[[u'10.20.10.1', u'001e-c1dc-fc01', u'N/A', u'M-GE0/0/0', u'12', u'D'],
 [u'10.20.1.254', u'001e-c1dc-fc01', u'1', u'XGE1/0/48', u'20', u'D'],
 [u'10.20.10.1', u'001e-c1dc-fc01', u'N/A', u'M-GE0/0/0', u'12', u'D'],
 [u'10.20.1.254', u'001e-c1dc-fc01', u'1', u'XGE1/0/48', u'20', u'D']]

In [13]:
for i in my_arps:
    print ("Mac address " + i[1] + ' has currently been assigned IP address ' + i[0] + ' and can be found on interface ' + i[3])

Mac address 001e-c1dc-fc01 has currently been assigned IP address 10.20.10.1 and can be found on interface M-GE0/0/0
Mac address 001e-c1dc-fc01 has currently been assigned IP address 10.20.1.254 and can be found on interface XGE1/0/48
Mac address 001e-c1dc-fc01 has currently been assigned IP address 10.20.10.1 and can be found on interface M-GE0/0/0
Mac address 001e-c1dc-fc01 has currently been assigned IP address 10.20.1.254 and can be found on interface XGE1/0/48
